**Please forgive some of the inefficient methods used!! Optimization takes more DSA knowledge than this!**

In [ ]:
import gurobipy as gp
import numpy as np

# **Task Graph Data Structure**

In [ ]:
class taskGraph:
    # graph defined by no.of nodes, wcets of nodes and adjaceny matrix
    def __init__(self,n,wcet,adj,m=1):
        self.n=n
        self.wcet=wcet
        self.adj=adj
        self.cores=m
        # in adjacency matrix, adj[i][j]=1 if there is a edge from i -> j

        # keep track of all the paths
        self.paths=[]

        #keep track of every parallel vertices for a given vertex
        self.asc=[]
        self.des=[]
        self.par=[]

        #len of longest path and volume of graph
        self.len=0
        self.vol=sum(self.wcet[i] for i in range(self.n))

        #length of longest path through vertex i
        self.l=[]

    def simplePL(self):
        p=[i for i in range(self.n)]
        def cmp(a,b):
            return self.l[a]>self.l[b] or (self.l[a]==self.l[b] and a<b)
        p.sort(key=lambda x: -self.l[x])
        return p

        

## Generate Random Graphs

In [ ]:
import networkx as nx
import numpy as np

def generate_random_dag_with_weights(num_vertices, num_edges):
    # Generate a random Directed Acyclic Graph
    dag = nx.gnm_random_graph(num_vertices, num_edges, directed=True)

    # Ensure the graph is a DAG
    while not nx.is_directed_acyclic_graph(dag):
        dag = nx.gnm_random_graph(num_vertices, num_edges, directed=True)

    # Add a single source and sink
    source = np.random.choice(list(dag.nodes))
    sink = np.random.choice(list(dag.nodes - set([source])))
    dag.add_edge(source, sink)

    # Assign random weights to each vertex
    weights = {node: np.random.randint(1, 10) for node in dag.nodes}
    nx.set_node_attributes(dag, weights, 'weight')

    return dag

def adjacency_matrix_with_weights_from_dag(dag):
    # Convert the DAG to an adjacency matrix with weights
    adjacency_matrix = nx.to_numpy_array(dag, dtype=int, weight='weight')

    return adjacency_matrix

# Example usage:

# Number of vertices and edges in the DAG
num_vertices = 8
num_edges = 12  # Adjust the number of edges as needed

# Generate a random DAG with a single source and sink and assign random weights
random_dag_with_weights = generate_random_dag_with_weights(num_vertices, num_edges)

# Generate and print the adjacency matrix with weights
adjacency_matrix_with_weights = adjacency_matrix_with_weights_from_dag(random_dag_with_weights)

print("Random DAG with Weights:")
print("Edges:", random_dag_with_weights.edges)
print("Node Weights:", nx.get_node_attributes(random_dag_with_weights, 'weight'))
print("\nAdjacency Matrix with Weights:")
print(adjacency_matrix_with_weights)

In [ ]:
n=num_vertices
wcet=np.random.rand(n)
adj=adjacency_matrix_with_weights

m=2
G=taskGraph(n,wcet,adj,m)

## Store the descendents of each vertex

In [ ]:
def find_des_util(G:taskGraph,i:int,n:int,se:set):
    if(i+1 in se):
        return
    for j in range(n):
        if(G.adj[i][j]==1):
            find_des_util(G,j,n,se)
            se.add(j+1)
    return

def find_des(G:taskGraph,i:int,n:int):
    se=set()
    find_des_util(G,i-1,n,se)
    return list(se)

G.des.clear()
for i in range(n):
    ls=find_des(G,i+1,n)
    G.des.append(ls.copy())
print(G.des)

## Store the ascendents of each vertex

In [ ]:
def find_asc_util(G:taskGraph,i:int,n:int,se:set):
    if(i+1 in se):
        return
    for j in range(n):
        if(G.adj[j][i]==1):
            find_asc_util(G,j,n,se)
            se.add(j+1)
    return

def find_asc(G:taskGraph,i:int,n:int):
    se=set()
    find_asc_util(G,i-1,n,se)
    return list(se)

G.asc.clear()
for i in range(n):
    ls=find_asc(G,i+1,n)
    G.asc.append(ls)

print(G.asc)

## Store parallel vertices of each vertex

In [ ]:
def parallel(G:taskGraph,i:int,n:int):
    ls=[]
    for j in range(n):
        if(i!=j and (j+1 not in G.des[i]) and (j+1 not in G.asc[i])):
            ls.append(j+1)
    return ls

G.par.clear()
for i in range(n):
    G.par.append(parallel(G,i,n).copy())

print(G.par)

## Create Paths Set ($\Pi$) and calculate the length of longest path ($len$)

In [ ]:
def pathset(G:taskGraph,i:int,n:int,path:list):
    k=0
    for j in range(n):
        if(G.adj[i][j]==1):
            cp=path.copy()
            cp.append(i+1)
            pathset(G,j,n,cp)
            k+=1
    if(k==0):
        cp=path.copy()
        cp.append(i+1)
        G.paths.append(cp)
    
pathset(G,0,n,[])

In [ ]:
print(G.paths)

In [ ]:
for path in G.paths:
    pathlength=0
    for node in path:
        pathlength+=G.wcet[node-1]
    G.len=max(G.len,pathlength)
print(G.len)

## Calculating the length of longest paths through each vertex $l(v_i)$

In [ ]:
for i in range(1,n+1):
    G.l.append(0)
    for path in G.paths:
        if(i in path):
            y=sum(G.wcet[j-1] for j in path)
            G.l[i-1]=max(G.l[i-1],y)

print(G.l)

## Calculating WCRT bound $R_P(G)$ for given priority assignment $P$

Please note that $R_P(G)$ is:

$$R_P(G) = \max_{\pi_k \in \Pi}len(\pi_k) + \frac{vol(I_p(\pi_k))}{m}$$

Since the formula for WCRT bound of a critical path $\pi_k$ under PLS algo with priority assignment $P$ is 

$$R_p(G,\pi_k) = \max_{\pi_k \in \Pi}len(\pi_k) + \frac{vol(I_p(\pi_k))}{m}$$

What we are doing is just iterating over all the "supposed" critical paths which is pretty damn inefficient

In [ ]:
P=[0,1,5,7,6,2,4,3]
Ipset=set()
for i in G.paths[2]:
    for j in G.par[i-1]:
        if(P[j-1]<P[i-1]):
            Ipset.add(j)
print(Ipset)

In [ ]:
def wcrt_inefficent(G:taskGraph,P:list):
    # function to compute the worst case response time given Priority
    R_max=0
    path_max=[]
    for path in G.paths:
        ## Calculate length of path
        length=sum(G.wcet[path[i]-1] for i in range(len(path)))

        ## Calculate the Interference set....
        Ip=set()
        for i in path:
            for j in G.par[i-1]:
                if(P[j-1]<P[i-1]):
                    Ip.add(j)

        Ip=list(Ip)        
        vol=sum(G.wcet[Ip[i]-1] for i in range(len(Ip)))

        R=length+(vol/G.cores)

        if(R>R_max):
            R_max=R
            path_max=path
        
    return R_max,path_max

In [ ]:
print(wcrt_inefficent(G,G.simplePL()))

In [ ]:
# initial priority assignment
print(G.simplePL())

#

# **ILP Solver**

## Laying down the formulation

- ***Objective Function***: The objective function we want to <u>**minimize**</u> is:
$R_P(G)$ for a given graph $G$ and varying over different priority assignments $P$ hence the problem being:
    $$\min_{P \in P(G)} R_P(G)$$

- ***Constraints***: 
    - $R$ constraints:
        - $R \geq  max\{len,\frac{vol}{m}\}$
        - $R \geq \sum_{v_i \in V} w_{i}^k  \;\;\; \forall \pi_k \in \Pi$

    - Weight constraints:
        - $w_{k}^i \geq c(v_i) \;\;\; \forall v_i \in \pi_k \;\;\; \forall \pi_k \in \Pi$   (For the nodes in critical path)
        - $w_{k}^i \geq 0 \;\;\; \forall v_i \in V$ 
        -  $\forall v_i \notin \pi_k, v_j \in par_k(v_i)\;\;\; \forall \pi_k \in \Pi$:
            - $w_{k}^i \geq \frac{c(v_i)}{m} - M \times y_{ij} ^ 1$
            - $p_i \leq p_j + M \times y_{ij}^1 $
            - $p_j < p_i + M \times y_{ij}^2$
            - $y_{ij}^1 + y_{ij}^2 = 1$
    

In [ ]:
opt_model=gp.Model(name="ILP_Solver")

### Primary Variables

In [ ]:
M=1e10

In [ ]:
p=opt_model.addVars(G.n,vtype=gp.GRB.INTEGER,lb=0,name="p")
R=opt_model.addVar(vtype=gp.GRB.CONTINUOUS,lb=max(G.len,G.vol/G.cores),name="R")

In [ ]:
y1=[]
y2=[]
for i in range(G.n):
    y1.append([])
    y2.append([])
    for j in range(G.n):
        y1[i].append(opt_model.addVar(vtype=gp.GRB.BINARY,name="y1_%d_%d" %(i,j)))
        y2[i].append(opt_model.addVar(vtype=gp.GRB.BINARY,name="y2_%d_%d" %(i,j)))

### Objective function Setup

In [ ]:
opt_model.update()
obj_fn=R
opt_model.setObjective(obj_fn,gp.GRB.MINIMIZE)

## Incremetal Update Algorithm

In [ ]:
wts=[]

In [ ]:
def inc_update_ilp(G:taskGraph):
    Pcb=None
    Rcb=1e10
     
    pi_dash=[]
    Pr=G.simplePL()     # done according to another paper. this method is just heuristic.
    Rn,pi=wcrt_inefficent(G,Pr)
    while(Rn<Rcb):
        RpG,pi=wcrt_inefficent(G,Pr)
        if(RpG<Rcb):
            Rcb=RpG
            Pcb=Pr
        
        if(pi in pi_dash):
            return Pcb,Rcb
        else:
            pi_dash.append(pi)

            wts.append([])
            k=len(wts)-1
            for i in range(G.n):
                wts[k].append(opt_model.addVar(vtype=gp.GRB.CONTINUOUS,lb=0,name='w_%d_%d'%(k,i)))

            opt_model.addConstr(R>=sum(wts[k][i] for i in range(G.n)))
            for i in range(G.n):
                if(i+1 in path):
                    opt_model.addConstr(wts[k][i]>=G.wcet[i])
                else:
                    for j in path:
                        if(j in G.par[i]):
                            eps=0.00001
                            opt_model.addConstr(wts[k][i]>=(G.wcet[i]/G.cores)-M*y1[i][j-1])
                            opt_model.addConstr(p[j-1]<=p[i]+M*y1[i][j-1])
                            opt_model.addConstr(p[i]<=p[j-1]+M*y2[i][j-1]-eps)
                            opt_model.addConstr(y1[i][j-1]+y2[i][j-1]==1)
            
            opt_model.update()
            opt_model.optimize()

            Rn=opt_model.objVal
            Pr=[opt_model.getVarByName("p[%d]" % i).x for i in range(G.n)]
    
    return Pcb,Rcb
            
                    


In [ ]:
print(inc_update_ilp(G))